# Mapping Snow Loss with Premade Datasets

In this example, we'll use `sankee` to visualize the conversion from snow and ice to barren and grassland on Baranof Island in Sitka, Alaska, as identified by the MODIS Land Cover dataset.

## Setup

In [1]:
import sankee
import ee

ee.Authenticate()
ee.Initialize()

## Premade Datasets

`sankee` plots are based on changes sampled randomly from a time series of classified images. You can use `sankee` with your own custom data, but it also includes premade classified datasets to get things up and running quickly. You can see the available datasets below.

In [2]:
sankee.datasets.datasets

[<Dataset: LCMS LC - Land Change Monitoring System Land Cover>,
 <Dataset: LCMS LU - Land Change Monitoring System Land Use>,
 <Dataset: NLCD - National Land Cover Database>,
 <Dataset: MCD12Q1 - MODIS Global Land Cover Type 1>,
 <Dataset: MCD12Q1 - MODIS Global Land Cover Type 2>,
 <Dataset: MCD12Q1 - MODIS Global Land Cover Type 3>,
 <Dataset: CGLS - Copernicus Global Land Cover>,
 <Dataset: C-CAP - NOAA Coastal Change Analysis Program 30m>,
 <Dataset: Canada Forested Ecosystem Land Cover>,
 <Dataset: LCMAP - Landscape Change Monitoring, Assessment, and Projection>,
 <Dataset: CORINE - Coordination of Information on the Environment>]

Each of the datasets above all of the parameters needed to create a Sankey diagram. For example, here are the class labels and colors for [LCMS Land Cover](https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2021-7).

In [3]:
sankee.datasets.LCMS_LC.df

,id,label,color
0,1,Trees,#005e00
1,2,Tall Shrubs & Trees Mix,#008000
2,3,Shrubs & Trees Mix,#00cc00
3,4,Grass/Forb/Herb & Trees Mix,#b3ff1a
4,5,Barren & Trees Mix,#99ff99
5,6,Tall Shrubs,#b30088
6,7,Shrubs,#e68a00
7,8,Grass/Forb/Herb & Shrubs Mix,#ffad33
8,9,Barren & Shrubs Mix,#ffe0b3
9,10,Grass/Forb/Herb,#ffff00


In this example, we'll use [MODIS Type 1 Land Cover](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1).

In [4]:
dataset = sankee.datasets.MODIS_LC_TYPE1

## Parameters

In order to build our Sankey diagram, we need to define a few parameters. First of all, what area are we interested in? This could be an image footprint, a fire perimeter, a buffered point, etc. For this example, we'll use a polygon from Baranof Island in Alaska.

In [5]:
aoi = ee.Geometry.Polygon([
    [-135.197519, 56.888425],
    [-134.774492, 56.888425],
    [-134.774492, 57.240839],
    [-135.197519, 57.240839],
])

The other required parameter is the year range. The available years will depend on the dataset (check them with `dataset.years`). We'll pick three, but you can technically pick as many as you want.

In [6]:
years = [2001, 2010, 2019]

### Plotting

That's all we need to generate a Sankey diagram. When we run `dataset.sankify`, random points will be sampled within our area of interest (adjust the number of points with `n`), and their land cover classification will be displayed over time. The initial plot looks pretty messy, so try turning classes on and off by clicking the color-coded buttons at the bottom!

In [7]:
dataset.sankify(years=years, region=aoi)

SankeyPlot()

If you turn off most of the smaller classes and the larger classes that didn't see much change, you can see how **permanent snow and ice** is being converted to **grassland** and **barren**. You can also hover over each link to see what percent of cover was changed in each time period.

In [8]:
dataset.sankify(years=years, region=aoi)

SankeyPlot()

## Mapping Images

To get a better look at the underlying data above, we can visualize the images using [geemap](https://github.com/giswqs/geemap).

In [ ]:
!pip install geemap

In [9]:
import geemap

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

`sankee` can retrieve a single year's image from a dataset using `get_year`. We'll use that to grab each year used in the Sankey diagram and add it to the map.

In [10]:
for year in years:
    img = dataset.get_year(year)
    Map.addLayer(img, {}, f"MODIS - {year}")

Map.addLayer(ee.Image().paint(aoi, 0, 4), {"palette": ["#fff"]}, "AOI")
Map.centerObject(aoi)

Click the images on and off in the interactive map to get a closer look at how the snow and ice extent shrunk over time.